In [2]:
import os
import sys

try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    current_script_dir = os.getcwd()

# Navigate up two levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)  # insert at the beginning to prioritize it


from src.analysis.decoding.decoding import plot_accuracies_with_multiple_sig_clusters
import pickle
import pandas as pd
import numpy as np



Qt5Agg backend not available, using default backend


Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


['/hpc/group/coganlab/jz421/GlobalLocal', '/hpc/home/jz421/miniconda3/envs/ieeg/lib/python311.zip', '/hpc/home/jz421/miniconda3/envs/ieeg/lib/python3.11', '/hpc/home/jz421/miniconda3/envs/ieeg/lib/python3.11/lib-dynload', '', '/hpc/home/jz421/miniconda3/envs/ieeg/lib/python3.11/site-packages', 'C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/']
Qt5Agg backend not available, using default backend


first load in the master results files

In [3]:
def load_master_results_file(base_path, epochs_root_file, master_results_file):
    with open(os.path.join(base_path, epochs_root_file, master_results_file), "rb") as f:
        master_results = pickle.load(f)
    return master_results

In [134]:
base_path = "/cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/decoding/figs"
epochs_root_file = "Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit"
congruency_by_switch_proportion_results_file = "20251106_081052_MASTER_RESULTS_16_subs_all_elecs_LDA_20boots_5splits_5reps_repeat_unit_ev_0.9.pkl"
switch_type_by_congruency_proportion_results_file = "20251105_223954_MASTER_RESULTS_16_subs_all_elecs_LDA_20boots_5splits_5reps_repeat_unit_ev_0.9.pkl"
lwpc_results_file = "20251105_224022_MASTER_RESULTS_16_subs_all_elecs_LDA_20boots_5splits_5reps_repeat_unit_ev_0.9.pkl"
lwps_results_file = "20251105_224057_MASTER_RESULTS_16_subs_all_elecs_LDA_20boots_5splits_5reps_repeat_unit_ev_0.9.pkl"

congruency_by_switch_proportion_results = load_master_results_file(base_path, epochs_root_file, congruency_by_switch_proportion_results_file)
switch_type_by_congruency_proportion_results = load_master_results_file(base_path, epochs_root_file, switch_type_by_congruency_proportion_results_file)
lwpc_results = load_master_results_file(base_path, epochs_root_file, lwpc_results_file)
lwps_results = load_master_results_file(base_path, epochs_root_file, lwps_results_file)


In [139]:
# --- Define Plotting Colors & Styles ---
lwpc_colors = {
    'c25_vs_i25': '#FF7E79', # Salmon
    'c75_vs_i75': '#DE8F05', # A warm Orange
    'chance': '#949494' # gray
}
lwpc_linestyles = {
    'c25_vs_i25': '-',  # Solid
    'c75_vs_i75': '-', # Solid
    'chance': '--' # dashed
}

# --- 3. Define Analysis Parameters ---
roi_to_plot = 'lpfc'
unit_of_analysis = lwpc_results['metadata']['args']['unit_of_analysis'] # 'repeat'
save_directory = os.path.join(base_path, epochs_root_file, f"{roi_to_plot}_poster_plots") 
os.makedirs(save_directory, exist_ok=True)

# --- 4. Extract Data for Plotting ---
print(f"Extracting data for ROI: {roi_to_plot}, Unit: {unit_of_analysis}")

time_points = lwpc_results['metadata']['time_window_centers']

# Get accuracy traces (the lines)
acc_25_vs_chance = lwpc_results['stats']['c25_vs_i25'][roi_to_plot][f'{unit_of_analysis}_true_accs']
acc_75_vs_chance = lwpc_results['stats']['c75_vs_i75'][roi_to_plot][f'{unit_of_analysis}_true_accs']
chance = lwpc_results['stats']['pooled_shuffles'][roi_to_plot]['lwpc']

# ⭐ CHANGE 2: Update the keys for the legend
accuracies_dict_to_plot = {
    '25% Inc. Prop.': acc_25_vs_chance,
    '75% Inc. Prop.': acc_75_vs_chance,
    'Chance': chance
}
# We also need to update the color/linestyle keys to match
final_colors = {
    '25% Inc. Prop.': lwpc_colors['c25_vs_i25'],
    '75% Inc. Prop.': lwpc_colors['c75_vs_i75'],
    'Chance': lwpc_colors['chance']
}
final_linestyles = {
    '25% Inc. Prop.': lwpc_linestyles['c25_vs_i25'],
    '75% Inc. Prop': lwpc_linestyles['c75_vs_i75'],
    'Chance': lwpc_linestyles['chance']
}

# Get significance masks (the bars)
sig_25_vs_chance = lwpc_results['stats']['c25_vs_i25'][roi_to_plot]['significant_clusters']
sig_75_vs_chance = lwpc_results['stats']['c75_vs_i75'][roi_to_plot]['significant_clusters']
comparison_clusters = lwpc_results['comparison_clusters'][roi_to_plot]['lwpc']
sig_25_over_75 = comparison_clusters['25_over_75']['clusters']
sig_75_over_25 = comparison_clusters['75_over_25']['clusters']
# --- Combine the two black-bar cluster masks ---
sig_comparison_combined = np.logical_or(sig_25_over_75, sig_75_over_25)

# --- 5. Build the Significance Dictionary ---
# ⭐ CHANGE 3: Update labels and colors here to match
significance_dict_to_plot = {
    # Bar 1 (Bottom): 25% vs Chance
    'sig_25_vs_chance': {
        'clusters': sig_25_vs_chance,
        'label': '25% I > Chance',
        'color': final_colors['25% Inc. Prop.'], # Salmon
        'marker': '' 
    },
    # Bar 2 (Middle): 75% vs Chance
    'sig_75_vs_chance': {
        'clusters': sig_75_vs_chance,
        'label': '75% I > Chance',
        'color': final_colors['75% Inc. Prop.'], # Orange
        'marker': '' 
    },
    # Bar 3 (Top): COMBINED Comparison
    'sig_comparison_combined': {
        'clusters': sig_comparison_combined,
        'label': '25% I > 75% I and 75% I > 25% I', # Combined label
        'color': 'black', 
        'marker': '',
        'level': 2  # Level 2
    }
}

# --- 6. Call the Plot Function ---
print("Generating plot...")
plot_accuracies_with_multiple_sig_clusters(
    time_points=time_points,
    accuracies_dict=accuracies_dict_to_plot,
    significance_clusters_dict=significance_dict_to_plot,
    
    # --- Aesthetics ---
    colors=final_colors,           # Use the renamed color dict
    linestyles=final_linestyles,   # Use the renamed linestyle dict
    ylabel="Congruency Decoding Accuracy",
    ylim=(0.3, 0.9), 
    
    # --- Key Changes ---
    show_chance_level=False, 
    show_sig_legend=False,    
    
    # --- Significance Bar Stacking ---
    sig_bar_base_position=0.87, 
    sig_bar_spacing=0.01,     
    sig_bar_height=0.005,       
    
    # --- File/Context Info ---
    comparison_name=f'LWPC_Comparison_Poster_Plot',
    roi=roi_to_plot,
    save_dir=save_directory,
    filename_suffix="poster_style_explicit_colors",
    
    # --- Other params ---
    window_size=lwpc_results['metadata']['args']['window_size'],
    step_size=lwpc_results['metadata']['args']['step_size'],
    sampling_rate=lwpc_results['metadata']['args']['sampling_rate'],
    single_column=True, 
    show_legend=False
)

print(f"✅ Plot saved in: {save_directory}")

Extracting data for ROI: lpfc, Unit: repeat
Generating plot...


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saved Nature-style plot with multiple significance clusters to: /cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/decoding/figs/Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit/lpfc_poster_plots/LWPC_Comparison_Poster_Plot_lpfc_poster_style_explicit_colors.pdf
✅ Plot saved in: /cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/decoding/figs/Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit/lpfc_poster_plots


In [140]:
# --- 2. Define Plotting Colors & Styles ---
lwps_colors = {
    's25_vs_r25': '#05B0F0', # Bright Cyan (Your choice)
    's75_vs_r75': '#0173B2', # Darker Blue
    'chance': '#949494'  # Gray
}
lwps_linestyles = {
    's25_vs_r25': '-',  # Solid
    's75_vs_r75': '-',  # Solid
    'chance': '--' # Dashed
}

# --- 3. Define Analysis Parameters ---
roi_to_plot = 'lpfc'
unit_of_analysis = lwps_results['metadata']['args']['unit_of_analysis'] # 'repeat'
save_directory = os.path.join(base_path, epochs_root_file, f"{roi_to_plot}_poster_plots") 
os.makedirs(save_directory, exist_ok=True)

# --- 4. Extract Data for Plotting ---
print(f"Extracting data for ROI: {roi_to_plot}, Unit: {unit_of_analysis}")

time_points = lwps_results['metadata']['time_window_centers']

# Get accuracy traces (the lines)
acc_25_vs_chance = lwps_results['stats']['s25_vs_r25'][roi_to_plot][f'{unit_of_analysis}_true_accs']
acc_75_vs_chance = lwps_results['stats']['s75_vs_r75'][roi_to_plot][f'{unit_of_analysis}_true_accs']
chance = lwps_results['stats']['pooled_shuffles'][roi_to_plot]['lwps']

# Update the keys for the legend
accuracies_dict_to_plot = {
    '25% Sw. Prop.': acc_25_vs_chance,
    '75% Sw. Prop.': acc_75_vs_chance,
    'Chance': chance,
}
# We also need to update the color/linestyle keys to match
final_colors = {
    '25% Sw. Prop.': lwps_colors['s25_vs_r25'],
    '75% Sw. Prop.': lwps_colors['s75_vs_r75'],
    'Chance': lwps_colors['chance'],
}
final_linestyles = {
    '25% Sw. Prop.': lwps_linestyles['s25_vs_r25'],
    '75% Sw. Prop.': lwps_linestyles['s75_vs_r75'],
    'Chance': lwps_linestyles['chance'],
}

# Get significance masks (the bars)
sig_25_vs_chance = lwps_results['stats']['s25_vs_r25'][roi_to_plot]['significant_clusters']
sig_75_vs_chance = lwps_results['stats']['s75_vs_r75'][roi_to_plot]['significant_clusters']
comparison_clusters = lwps_results['comparison_clusters'][roi_to_plot]['lwps']
sig_25_over_75 = comparison_clusters['25_over_75']['clusters']
sig_75_over_25 = comparison_clusters['75_over_25']['clusters']
sig_comparison_combined = np.logical_or(sig_25_over_75, sig_75_over_25)

# --- 5. Build the Significance Dictionary ---
significance_dict_to_plot = {
    # Bar 1 (Bottom): 25% vs Chance
    'sig_25_vs_chance': {
        'clusters': sig_25_vs_chance,
        'label': '25% S > Chance',
        'color': final_colors['25% Sw. Prop.'], # Bright Cyan
        'marker': '' 
    },
    # Bar 2 (Middle): 75% vs Chance
    'sig_75_vs_chance': {
        'clusters': sig_75_vs_chance,
        'label': '75% S > Chance',
        'color': final_colors['75% Sw. Prop.'], # Dark Blue
        'marker': '' 
    },
    # Bar 3 (Top): COMBINED Comparison
    'sig_comparison_combined': {
        'clusters': sig_comparison_combined,
        'label': '25% S > 75% S and 75% S > 25% S', # Combined label
        'color': 'black', 
        'marker': '',
        'level': 2  # Level 2
    }
}

# --- 6. Call the Plot Function ---
print("Generating plot...")
plot_accuracies_with_multiple_sig_clusters(
    time_points=time_points,
    accuracies_dict=accuracies_dict_to_plot,
    significance_clusters_dict=significance_dict_to_plot,
    
    # --- Aesthetics ---
    colors=final_colors,
    linestyles=final_linestyles,
    ylabel="Switch Type Decoding Accuracy", # CHANGED Y-label
    ylim=(0.3, 0.9), 
    
    # --- Key Changes ---
    show_chance_level=False, # We plot chance as a trace
    show_legend=False,        
    show_sig_legend=False,    
    
    # --- Significance Bar Stacking ---
    sig_bar_base_position=0.87, 
    sig_bar_spacing=0.01,
    sig_bar_height=0.005,
    
    # --- File/Context Info ---
    comparison_name=f'LWPS_Comparison_Poster_Plot', # CHANGED comparison name
    roi=roi_to_plot,
    save_dir=save_directory,
    filename_suffix="poster_style_explicit_colors_with_chance",
    
    # --- Other params ---
    window_size=lwps_results['metadata']['args']['window_size'],
    step_size=lwps_results['metadata']['args']['step_size'],
    sampling_rate=lwps_results['metadata']['args']['sampling_rate'],
    single_column=True 
)

print(f"✅ Plot saved in: {save_directory}")

Extracting data for ROI: lpfc, Unit: repeat
Generating plot...


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saved Nature-style plot with multiple significance clusters to: /cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/decoding/figs/Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit/lpfc_poster_plots/LWPS_Comparison_Poster_Plot_lpfc_poster_style_explicit_colors_with_chance.pdf
✅ Plot saved in: /cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/decoding/figs/Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit/lpfc_poster_plots


congruency by switch prop

In [142]:
# --- 2. Define Plotting Colors & Styles (Using LWPS Blue Scheme) ---
congruency_by_switch_proportion_colors = {
    'c_in_25switchBlock_vs_i_in_25switchBlock': '#05B0F0', # Bright Cyan
    'c_in_75switchBlock_vs_i_in_75switchBlock': '#0173B2', # Darker Blue
    'pooled_chance': '#949494'  # Gray
}
congruency_by_switch_proportion_linestyles = {
    'c_in_25switchBlock_vs_i_in_25switchBlock': '-',  # Solid
    'c_in_75switchBlock_vs_i_in_75switchBlock': '-',  # Solid
    'pooled_chance': '--' # Dashed
}

# --- 3. Define Analysis Parameters ---
roi_to_plot = 'lpfc'
unit_of_analysis = congruency_by_switch_proportion_results['metadata']['args']['unit_of_analysis'] # 'repeat'
save_directory = os.path.join(base_path, epochs_root_file, f"{roi_to_plot}_poster_plots") 
os.makedirs(save_directory, exist_ok=True)

# --- 4. Extract Data for Plotting ---
print(f"Extracting data for ROI: {roi_to_plot}, Unit: {unit_of_analysis}")

time_points = congruency_by_switch_proportion_results['metadata']['time_window_centers']

# Get accuracy traces (the lines)
acc_25_vs_chance = congruency_by_switch_proportion_results['stats']['c_in_25switchBlock_vs_i_in_25switchBlock'][roi_to_plot][f'{unit_of_analysis}_true_accs']
acc_75_vs_chance = congruency_by_switch_proportion_results['stats']['c_in_75switchBlock_vs_i_in_75switchBlock'][roi_to_plot][f'{unit_of_analysis}_true_accs']
pooled_chance_acc = congruency_by_switch_proportion_results['stats']['pooled_shuffles'][roi_to_plot]['congruency_by_switch_proportion']

# Update the keys for the legend
accuracies_dict_to_plot = {
    '25% Sw. Block (C/I Dec.)': acc_25_vs_chance,
    '75% Sw. Block (C/I Dec.)': acc_75_vs_chance,
    'Pooled Chance': pooled_chance_acc,
}

# We also need to update the color/linestyle keys to match
final_colors = {
    '25% Sw. Block (C/I Dec.)': congruency_by_switch_proportion_colors['c_in_25switchBlock_vs_i_in_25switchBlock'],
    '75% Sw. Block (C/I Dec.)': congruency_by_switch_proportion_colors['c_in_75switchBlock_vs_i_in_75switchBlock'],
    'Pooled Chance': congruency_by_switch_proportion_colors['pooled_chance'],
}
final_linestyles = {
    '25% Sw. Block (C/I Dec.)': congruency_by_switch_proportion_linestyles['c_in_25switchBlock_vs_i_in_25switchBlock'],
    '75% Sw. Block (C/I Dec.)': congruency_by_switch_proportion_linestyles['c_in_75switchBlock_vs_i_in_75switchBlock'],
    'Pooled Chance': congruency_by_switch_proportion_linestyles['pooled_chance'],
}

# Get significance masks (the bars)
sig_25_vs_chance = congruency_by_switch_proportion_results['stats']['c_in_25switchBlock_vs_i_in_25switchBlock'][roi_to_plot]['significant_clusters']
sig_75_vs_chance = congruency_by_switch_proportion_results['stats']['c_in_75switchBlock_vs_i_in_75switchBlock'][roi_to_plot]['significant_clusters']
comparison_clusters = congruency_by_switch_proportion_results['comparison_clusters'][roi_to_plot]['congruency_by_switch_proportion']
sig_25_over_75 = comparison_clusters['25_over_75']['clusters']
sig_75_over_25 = comparison_clusters['75_over_25']['clusters']
# --- Combine the two black-bar cluster masks ---
sig_comparison_combined = np.logical_or(sig_25_over_75, sig_75_over_25)

# --- 5. Build the Significance Dictionary ---
significance_dict_to_plot = {
    # Bar 1 (Bottom): 25% vs Chance
    'sig_25_vs_chance': {
        'clusters': sig_25_vs_chance,
        'label': 'C/I (25S) > Chance',
        'color': final_colors['25% Sw. Block (C/I Dec.)'], # Bright Cyan
        'marker': '' 
    },
    # Bar 2 (Middle): 75% vs Chance
    'sig_75_vs_chance': {
        'clusters': sig_75_vs_chance,
        'label': 'C/I (75S) > Chance',
        'color': final_colors['75% Sw. Block (C/I Dec.)'], # Dark Blue
        'marker': '' 
    },
    # Bar 3 (Top): COMBINED Comparison
    'sig_comparison_combined': {
        'clusters': sig_comparison_combined,
        'label': '25% S > 75% S and 75% S > 25% S', # Combined label
        'color': 'black', 
        'marker': '',
        'level': 2  # Level 2
    }
}

# --- 6. Call the Plot Function ---
print("Generating plot...")
plot_accuracies_with_multiple_sig_clusters(
    time_points=time_points,
    accuracies_dict=accuracies_dict_to_plot,
    significance_clusters_dict=significance_dict_to_plot,
    
    # --- Aesthetics ---
    colors=final_colors,
    linestyles=final_linestyles,
    ylabel="Congruency Decoding Accuracy", 
    ylim=(0.3, 0.9), 
    
    # --- Key Changes ---
    show_chance_level=False, # We plot chance as a trace
    show_legend=False,        
    show_sig_legend=False,    
    
    # --- Significance Bar Stacking ---
    sig_bar_base_position=0.87, 
    sig_bar_spacing=0.01,
    sig_bar_height=0.005,
    
    # --- File/Context Info ---
    comparison_name=f'congruency_by_switch_proportion_comparison_Poster_Plot',
    roi=roi_to_plot,
    save_dir=save_directory,
    filename_suffix="poster_style_congruency_by_switch_proportion_with_chance",
    
    # --- Other params ---
    window_size=congruency_by_switch_proportion_results['metadata']['args']['window_size'],
    step_size=congruency_by_switch_proportion_results['metadata']['args']['step_size'],
    sampling_rate=congruency_by_switch_proportion_results['metadata']['args']['sampling_rate'],
    single_column=True 
)

print(f"✅ Plot saved in: {save_directory}")

Extracting data for ROI: lpfc, Unit: repeat
Generating plot...


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saved Nature-style plot with multiple significance clusters to: /cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/decoding/figs/Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit/lpfc_poster_plots/congruency_by_switch_proportion_comparison_Poster_Plot_lpfc_poster_style_congruency_by_switch_proportion_with_chance.pdf
✅ Plot saved in: /cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/decoding/figs/Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit/lpfc_poster_plots


switch type by con prop

In [ ]:

# --- 2. Define Plotting Colors & Styles (Using LWPC Red/Orange Scheme) ---
switch_type_by_congruency_proportion_colors = {
    's_in_25incongruentBlock_vs_r_in_25incongruentBlock': '#FF7E79', # Salmon
    's_in_75incongruentBlock_vs_r_in_75incongruentBlock': '#DE8F05', # A warm Orange
    'pooled_chance': '#949494' # gray
}
switch_type_by_congruency_proportion_linestyles = {
    's_in_25incongruentBlock_vs_r_in_25incongruentBlock': '-',  # Solid
    's_in_75incongruentBlock_vs_r_in_75incongruentBlock': '-', # Solid
    'pooled_chance': '--' # dashed
}

# --- 3. Define Analysis Parameters ---
roi_to_plot = 'lpfc'
unit_of_analysis = switch_type_by_congruency_proportion_results['metadata']['args']['unit_of_analysis'] # 'repeat'
save_directory = os.path.join(base_path, epochs_root_file, f"{roi_to_plot}_poster_plots") 
os.makedirs(save_directory, exist_ok=True)

# --- 4. Extract Data for Plotting ---
print(f"Extracting data for ROI: {roi_to_plot}, Unit: {unit_of_analysis}")

time_points = switch_type_by_congruency_proportion_results['metadata']['time_window_centers']

# Get accuracy traces (the lines)
acc_25_vs_chance = switch_type_by_congruency_proportion_results['stats']['s_in_25incongruentBlock_vs_r_in_25incongruentBlock'][roi_to_plot][f'{unit_of_analysis}_true_accs']
acc_75_vs_chance = switch_type_by_congruency_proportion_results['stats']['s_in_75incongruentBlock_vs_r_in_75incongruentBlock'][roi_to_plot][f'{unit_of_analysis}_true_accs']
chance = switch_type_by_congruency_proportion_results['stats']['pooled_shuffles'][roi_to_plot]['switch_type_by_congruency_proportion']

# Update the keys for the legend
accuracies_dict_to_plot = {
    '25% Inc. Block (S/R Dec.)': acc_25_vs_chance,
    '75% Inc. Block (S/R Dec.)': acc_75_vs_chance,
    'Chance': chance
}
# We also need to update the color/linestyle keys to match
final_colors = {
    '25% Inc. Block (S/R Dec.)': switch_type_by_congruency_proportion_colors['s_in_25incongruentBlock_vs_r_in_25incongruentBlock'],
    '75% Inc. Block (S/R Dec.)': switch_type_by_congruency_proportion_colors['s_in_75incongruentBlock_vs_r_in_75incongruentBlock'],
    'Chance': switch_type_by_congruency_proportion_colors['pooled_chance']
}
final_linestyles = {
    '25% Inc. Block (S/R Dec.)': switch_type_by_congruency_proportion_linestyles['s_in_25incongruentBlock_vs_r_in_25incongruentBlock'],
    '75% Inc. Block (S/R Dec.)': switch_type_by_congruency_proportion_linestyles['s_in_75incongruentBlock_vs_r_in_75incongruentBlock'],
    'Chance': switch_type_by_congruency_proportion_linestyles['pooled_chance']
}

# Get significance masks (the bars)
sig_25_vs_chance = switch_type_by_congruency_proportion_results['stats']['s_in_25incongruentBlock_vs_r_in_25incongruentBlock'][roi_to_plot]['significant_clusters']
sig_75_vs_chance = switch_type_by_congruency_proportion_results['stats']['s_in_75incongruentBlock_vs_r_in_75incongruentBlock'][roi_to_plot]['significant_clusters']
comparison_clusters = switch_type_by_congruency_proportion_results['comparison_clusters'][roi_to_plot]['switch_type_by_congruency_proportion']
sig_25_over_75 = comparison_clusters['25_over_75']['clusters']
sig_75_over_25 = comparison_clusters['75_over_25']['clusters']

# --- Combine the two black-bar cluster masks ---
sig_comparison_combined = np.logical_or(sig_25_over_75, sig_75_over_25)

# --- 5. Build the Significance Dictionary ---
significance_dict_to_plot = {
    # Bar 1 (Bottom): 25% vs Chance
    'sig_25_vs_chance': {
        'clusters': sig_25_vs_chance,
        'label': 'S/R (25I) > Chance',
        'color': final_colors['25% Inc. Block (S/R Dec.)'], # Salmon
        'marker': '',
        'level': 0  # Level 0
    },
    # Bar 2 (Middle): 75% vs Chance
    'sig_75_vs_chance': {
        'clusters': sig_75_vs_chance,
        'label': 'S/R (75I) > Chance',
        'color': final_colors['75% Inc. Block (S/R Dec.)'], # Orange
        'marker': '',
        'level': 1  # Level 1
    },
    # Bar 3 (Top): COMBINED Comparison
    'sig_comparison_combined': {
        'clusters': sig_comparison_combined,
        'label': 'S/R (25I) vs S/R (75I)', # Combined label
        'color': 'black', 
        'marker': '',
        'level': 2  # Level 2
    }
}

# --- 6. Call the Plot Function ---
print("Generating plot...")
plot_accuracies_with_multiple_sig_clusters(
    time_points=time_points,
    accuracies_dict=accuracies_dict_to_plot,
    significance_clusters_dict=significance_dict_to_plot,
    
    # --- Aesthetics ---
    colors=final_colors,           # Use the renamed color dict
    linestyles=final_linestyles,   # Use the renamed linestyle dict
    ylabel="Switch Type Decoding Accuracy",
    ylim=(0.3, 0.9), 
    
    # --- Key Changes ---
    show_chance_level=False, # We plot chance as a trace
    show_legend=False,        
    show_sig_legend=False,    
    
    # --- Significance Bar Stacking ---
    sig_bar_base_position=0.87, 
    sig_bar_spacing=0.01,
    sig_bar_height=0.005,
    
    # --- File/Context Info ---
    comparison_name=f'switch_type_by_congruency_proportion_Comparison_Poster_Plot',
    roi=roi_to_plot,
    save_dir=save_directory,
    filename_suffix="poster_style_switch_type_by_congruency_proportion_with_chance",
    
    # --- Other params ---
    window_size=switch_type_by_congruency_proportion_results['metadata']['args']['window_size'],
    step_size=switch_type_by_congruency_proportion_results['metadata']['args']['step_size'],
    sampling_rate=switch_type_by_congruency_proportion_results['metadata']['args']['sampling_rate'],
    single_column=True 
)

print(f"✅ Plot saved in: {save_directory}")

Extracting data for ROI: lpfc, Unit: repeat
Generating plot...


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Saved Nature-style plot with multiple significance clusters to: /cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/decoding/figs/Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit/lpfc_poster_plots/switch_type_by_congruency_proportion_Comparison_Poster_Plot_lpfc_poster_style_switch_type_by_congruency_proportion_with_chance.pdf
✅ Plot saved in: /cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/decoding/figs/Stimulus_0.5sec_within-1.0-0.0sec_base_decFactor_8_outliers_10_drop_thresh_perc_5.0_70.0-150.0_Hz_padLength_0.5s_stat_func_ttest_ind_equal_var_False_nan_policy_omit/lpfc_poster_plots


if doing err corr

In [4]:
base_path = base_path = "/cwork/jz421/BIDS-1.1_GlobalLocal/BIDS/derivatives/decoding/figs"
epochs_root_file = "Stimulus_0.5sec_within-1-0sec_randoffset_StimulusBase_decFactor_8_markOutliersAsNaN_False_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
err_corr_results_file = "20251108_192052_MASTER_RESULTS_13_subs_all_elecs_LDA_20boots_5splits_5reps_repeat_unit_ev_0.9.pkl"
is_cr_err_corr_results_file = '20251108_192052_MASTER_RESULTS_6_subs_all_elecs_LDA_20boots_5splits_5reps_repeat_unit_ev_0.9.pkl'

err_corr_results = load_master_results_file(base_path, epochs_root_file, err_corr_results_file)
is_cr_err_corr_results = load_master_results_file(base_path, epochs_root_file, is_cr_err_corr_results_file)


In [7]:
is_cr_err_corr_results['stats']

{}

In [5]:
err_corr_results

{'stats': {'err_vs_corr': {'lpfc': {'repeat_true_accs': array([[0.51999998, 0.51999998, 0.5       , ..., 0.80000001, 0.83999997,
            0.81999999],
           [0.38      , 0.41999999, 0.34      , ..., 0.74000001, 0.75999999,
            0.83999997],
           [0.51999998, 0.51999998, 0.47999999, ..., 0.75999999, 0.83999997,
            0.83999997],
           ...,
           [0.5       , 0.5       , 0.5       , ..., 0.72000003, 0.83999997,
            0.88      ],
           [0.5       , 0.5       , 0.57999998, ..., 0.68000001, 0.80000001,
            0.77999997],
           [0.5       , 0.5       , 0.54000002, ..., 0.69999999, 0.81999999,
            0.86000001]], shape=(100, 37)),
    'mean_true_across_repeats': array([[0.467     , 0.4506    , 0.4874    , 0.51      , 0.5024    ,
            0.4816    , 0.4698    , 0.4572    , 0.4858    , 0.5322    ,
            0.5838    , 0.5914    , 0.553     , 0.5398    , 0.521     ,
            0.5242    , 0.5432    , 0.5398    , 0.5468   

In [ ]:
# --- 2. Define Plotting Colors & Styles (Using LWPC Red/Orange Scheme) ---
err_corr_colors = {
    'err_vs_corr': 'black', # Salmon
    'pooled_chance': '#949494' # gray
}
err_corr_linestyles = {
    'err_vs_corr': '-',  # Solid
    'pooled_chance': '--' # dashed
}

# --- 3. Define Analysis Parameters ---
roi_to_plot = 'lpfc'
unit_of_analysis = err_corr_results['metadata']['args']['unit_of_analysis'] # 'repeat'
save_directory = os.path.join(base_path, epochs_root_file, f"{roi_to_plot}_poster_plots") 
os.makedirs(save_directory, exist_ok=True)

# --- 4. Extract Data for Plotting ---
print(f"Extracting data for ROI: {roi_to_plot}, Unit: {unit_of_analysis}")

time_points = err_corr_results['metadata']['time_window_centers']

# Get accuracy traces (the lines)
acc_25_vs_chance = err_corr_results['stats']['err_vs_corr'][roi_to_plot][f'{unit_of_analysis}_true_accs']
acc_75_vs_chance = err_corr_results['stats']['s_in_75incongruentBlock_vs_r_in_75incongruentBlock'][roi_to_plot][f'{unit_of_analysis}_true_accs']
chance = switch_type_by_congruency_proportion_results['stats']['pooled_shuffles'][roi_to_plot]['switch_type_by_congruency_proportion']

# Update the keys for the legend
accuracies_dict_to_plot = {
    '25% Inc. Block (S/R Dec.)': acc_25_vs_chance,
    '75% Inc. Block (S/R Dec.)': acc_75_vs_chance,
    'Chance': chance
}
# We also need to update the color/linestyle keys to match
final_colors = {
    '25% Inc. Block (S/R Dec.)': switch_type_by_congruency_proportion_colors['s_in_25incongruentBlock_vs_r_in_25incongruentBlock'],
    '75% Inc. Block (S/R Dec.)': switch_type_by_congruency_proportion_colors['s_in_75incongruentBlock_vs_r_in_75incongruentBlock'],
    'Chance': switch_type_by_congruency_proportion_colors['pooled_chance']
}
final_linestyles = {
    '25% Inc. Block (S/R Dec.)': switch_type_by_congruency_proportion_linestyles['s_in_25incongruentBlock_vs_r_in_25incongruentBlock'],
    '75% Inc. Block (S/R Dec.)': switch_type_by_congruency_proportion_linestyles['s_in_75incongruentBlock_vs_r_in_75incongruentBlock'],
    'Chance': switch_type_by_congruency_proportion_linestyles['pooled_chance']
}

# Get significance masks (the bars)
sig_25_vs_chance = switch_type_by_congruency_proportion_results['stats']['s_in_25incongruentBlock_vs_r_in_25incongruentBlock'][roi_to_plot]['significant_clusters']
sig_75_vs_chance = switch_type_by_congruency_proportion_results['stats']['s_in_75incongruentBlock_vs_r_in_75incongruentBlock'][roi_to_plot]['significant_clusters']
comparison_clusters = switch_type_by_congruency_proportion_results['comparison_clusters'][roi_to_plot]['switch_type_by_congruency_proportion']
sig_25_over_75 = comparison_clusters['25_over_75']['clusters']
sig_75_over_25 = comparison_clusters['75_over_25']['clusters']

# --- Combine the two black-bar cluster masks ---
sig_comparison_combined = np.logical_or(sig_25_over_75, sig_75_over_25)

# --- 5. Build the Significance Dictionary ---
significance_dict_to_plot = {
    # Bar 1 (Bottom): 25% vs Chance
    'sig_25_vs_chance': {
        'clusters': sig_25_vs_chance,
        'label': 'S/R (25I) > Chance',
        'color': final_colors['25% Inc. Block (S/R Dec.)'], # Salmon
        'marker': '',
        'level': 0  # Level 0
    },
    # Bar 2 (Middle): 75% vs Chance
    'sig_75_vs_chance': {
        'clusters': sig_75_vs_chance,
        'label': 'S/R (75I) > Chance',
        'color': final_colors['75% Inc. Block (S/R Dec.)'], # Orange
        'marker': '',
        'level': 1  # Level 1
    },
    # Bar 3 (Top): COMBINED Comparison
    'sig_comparison_combined': {
        'clusters': sig_comparison_combined,
        'label': 'S/R (25I) vs S/R (75I)', # Combined label
        'color': 'black', 
        'marker': '',
        'level': 2  # Level 2
    }
}

# --- 6. Call the Plot Function ---
print("Generating plot...")
plot_accuracies_with_multiple_sig_clusters(
    time_points=time_points,
    accuracies_dict=accuracies_dict_to_plot,
    significance_clusters_dict=significance_dict_to_plot,
    
    # --- Aesthetics ---
    colors=final_colors,           # Use the renamed color dict
    linestyles=final_linestyles,   # Use the renamed linestyle dict
    ylabel="Switch Type Decoding Accuracy",
    ylim=(0.3, 0.9), 
    
    # --- Key Changes ---
    show_chance_level=False, # We plot chance as a trace
    show_legend=False,        
    show_sig_legend=False,    
    
    # --- Significance Bar Stacking ---
    sig_bar_base_position=0.87, 
    sig_bar_spacing=0.01,
    sig_bar_height=0.005,
    
    # --- File/Context Info ---
    comparison_name=f'switch_type_by_congruency_proportion_Comparison_Poster_Plot',
    roi=roi_to_plot,
    save_dir=save_directory,
    filename_suffix="poster_style_switch_type_by_congruency_proportion_with_chance",
    
    # --- Other params ---
    window_size=switch_type_by_congruency_proportion_results['metadata']['args']['window_size'],
    step_size=switch_type_by_congruency_proportion_results['metadata']['args']['step_size'],
    sampling_rate=switch_type_by_congruency_proportion_results['metadata']['args']['sampling_rate'],
    single_column=True 
)

print(f"✅ Plot saved in: {save_directory}")